In [1]:
import pandas as pd
import numpy as np
import os
import datetime
import time
import json
import requests
from facebookads.api import FacebookAdsApi
from facebookads.adobjects.adset import AdSet
from facebookads.adobjects.ad import Ad
from facebookads.adobjects.adaccount import AdAccount
from facebookads.adobjects.adcreative import AdCreative
from facebookads.adobjects.customaudience import CustomAudience
from facebookads.adobjects.campaign import Campaign

jian_token=json.load(open("/mnt/clients/juba/hqjubaapp02/sharefolder/Docs/FB_token/lasest_FB_token_jian.json","r"))['latest_FB_token']

BigLotsAccount_id="act_271491453638620"
BigLotsDcart10_id="act_2449979771956630"
list_all_fb_account_ids=[BigLotsAccount_id,BigLotsDcart10_id]
APP_ID='1954772121206438'
API_VERSION = 'v8.0'
folder_archieved_list="/mnt/clients/juba/hqjubaapp02/sharefolder/Docs/Audience_uploading_scripts/Facebook/Audience_name_list_On_App/archived_audience_list/"
latest_json_list="/mnt/clients/juba/hqjubaapp02/sharefolder/Docs/Audience_uploading_scripts/Facebook/Audience_name_list_On_App/latest_update.json"
latest_category_mapping="/mnt/clients/juba/hqjubaapp02/sharefolder/Docs/Audience_uploading_scripts/Facebook/Audience_name_list_On_App/latest_category_mapping.json"
FacebookAdsApi.init(access_token=jian_token, api_version=API_VERSION)

folder_csv_output="/mnt/clients/juba/hqjubaapp02/sharefolder/Docs/Audience_uploading_scripts/Facebook/Audience_name_list_On_App/output_csv_files/"

In [2]:
# Due to the new audience names needed to be updated, making no sense to run both account together
# unless all new audience are totaly duplicated with each other
# For now, run seperately

dict_updated_json_config={}
dict_previous_json_config=json.load(open(latest_json_list,'r'))
def func_fetch_df_names(bl_fb_acctount_id):
    myAccount = AdAccount(BigLotsAccount_id)
    custom_audiences = myAccount.get_custom_audiences(fields=['name'])
    df = pd.DataFrame(custom_audiences)
    return df

In [3]:
dict_previous_json_config[BigLotsDcart10_id]=dict_previous_json_config[BigLotsAccount_id]

In [4]:
# dict_previous_json_config=json.load(open(latest_json_list,'r'))
for i in list_all_fb_account_ids:
    dict_previous_json_config[i].update({'date_archived_on':str(datetime.datetime.now().date())})

# BigLotsAccount_id="act_271491453638620"

In [5]:
# 1: BigLotsAccount_id="act_271491453638620"
previous_list=dict_previous_json_config[BigLotsAccount_id]['aud_list']
previous_list.sort()

df=func_fetch_df_names(BigLotsAccount_id)


In [6]:
df.head(3)

,id,name
0,23846130655090265,aud_12162020_PYD_Active_MS_Hashed_20201215020409
1,23846093341540265,AgilOne_121120 Weekly Core Rewards FB
2,23846093340280265,AgilOne_121120 Weekly Furniture Rewards FB


In [7]:
previous_list.sort()
previous_list

['BL_Mobile_20200616-20200715_Non-furniture Specific Competitors_Exposed_20200727',
 'BL_Mobile_20200616-20200715_Vicinity_Exposed_20200727',
 'BL_Mobile_20200616-20200715_Visitors_Exposed_20200727',
 'BL_Mobile_20200616-20200715_Vulnerable Competitors_Exposed_20200727',
 'Lookalike (US, 1%) - Big Lots - US (90 days) Store Visit - MS',
 'Lookalike (US, 1%) - Store Visits - 90 days',
 'Lookalike (US, 1%) - T_Christmas_360_Active',
 'Lookalike (US, 1%) - T_Christmas_360_Lapsed',
 'Lookalike (US, 1%) - T_Seasonal362363_Active',
 'Lookalike (US, 1%) - T_Seasonal362363_Lapsed',
 'Lookalike (US, 5%) - Offline Purchasers last 90 days',
 'Lookalike (US, 5%) - Purchase Pixel - 180 Days',
 'Lookalike (US, 8%) - Offline Purchasers last 90 days',
 'Lookalike (US, 8%) - Purchase Pixel - 180 Days',
 'Sign_Up_No_Purchase_7_to_21_Days_Test_NEW',
 'T_Christmas_360_Active',
 'T_Christmas_360_Lapsed',
 'T_Seasonal362363_Active',
 'T_Seasonal362363_Lapsed',
 'Website Visitors - Past 180 Days',
 'aud_10-12

In [8]:
list_to_remove=[]
#
list_to_remove_1=[x for x in previous_list if "T_aud_21_goal_" in x]
list_to_remove.extend(list_to_remove_1)
#
list_to_remove_2=[x for x in previous_list if "2020Q3" in x]
list_to_remove.extend(list_to_remove_2)
list_to_remove

[]

In [9]:
for i in list_to_remove:
    try:
        previous_list.remove(i)
        print("%s removed from previous_list"%i)
    except:
        print("%s not in previous_list"%i)
        pass
    

In [10]:
list_all_aud=df['name'].tolist()

In [11]:
list_new_names=[x for x in list_all_aud if "12162020_PYD_Active_MS_Hashed_20201215020409" in x] # give the pattern and do filter

list_new_names.sort()

print(list_new_names)


['aud_12162020_PYD_Active_MS_Hashed_20201215020409']


In [12]:
print(len(list_new_names))

1


In [13]:
list_updated=list(set(previous_list+list_new_names))
dict_account_1_updated={BigLotsAccount_id:{'date_update_on':str(datetime.datetime.now().date()),'aud_list':list_updated}}
df=df[df['name'].isin(list_updated)]
len(list_updated),len(df.index)

(44, 44)

In [14]:
# Function here and the rename below is hard-coded, so the 2 accounts are done in differnt cells not in loop

# list_n contain the audience names on the FB UI
# names_n (dict_n.keys()) is the new names on the Juba App (can contain 1 to many FB audience based on request)

def func_categorize_aud(df_FB_aud_id_name):
    list_names=df_FB_aud_id_name['name'].tolist()
    df_FB_aud_id_name=df_FB_aud_id_name.rename(columns={"name":"orgname"})
    dict_overall={}
    
    # 0 new 
    list_0_new_org=[x for x in list_names if 'aud_T_' in x]
    list_0_new_renamed=["2020Q4_Rewards_"+x for x in list_0_new_org]    
    dict_0=dict(zip(list_0_new_org,list_0_new_renamed))
    dict_overall.update(dict_0)
    
    # 1 LAL -- requested from Spencer for Seasonal audience look alike will be updated later
    list_1=[x for x in list_names if 'Lookalike' in x]
    dict_1=dict(zip(list_1,['LAL']*len(list_1)))
    dict_overall.update(dict_1)
    
    # 2 Mobile Visitors
    list_2=[x for x in list_names if 'Mobile' in x and 'Visitors' in x]
    dict_2=dict(zip(list_2,['Mobile Visitors']*len(list_2)))
    dict_overall.update(dict_2)
    
    # 3 Mobile Vicinity
    list_3=[x for x in list_names if 'Mobile' in x and 'Vicinity' in x]
    dict_3=dict(zip(list_3,['Mobile Vicinity']*len(list_3)))
    
    # 4 Mobile Non-furniture Specific Competitors
    list_4=[x for x in list_names if 'Mobile' in x and 'Non-furniture Specific Competitors' in x]
    dict_4=dict(zip(list_4,['Mobile Non-furniture Specific Competitors']*len(list_4)))    
    
    # 5 Mobile Vulnerable Competitors
    list_5=[x for x in list_names if 'Mobile' in x and 'Vulnerable Competitors' in x]
    dict_5=dict(zip(list_5,['Mobile Vulnerable Competitors']*len(list_5)))
    
    # 6 Retargeting
    list_6=[x for x in list_names if 'Website Visitors' in x]
    dict_6=dict(zip(list_6,['Retargeting']*len(list_6)))
    
    # 7 SOTF_NA_Rewards_Lapsed1318_NA
    list_7=[x for x in list_names if 'SOTF' in x and 'Lapsed1318' in x]
    list_7=[x for x in list_7 if x not in list_0_new_org]
    dict_7=dict(zip(list_7,['SOTF_NA_Rewards_Lapsed1318_NA']*len(list_7)))
    
    # 8 SOTF_NA_Rewards_Active012_NA
    list_8=[x for x in list_names if 'SOTF' in x and 'Active' in x]
    list_8=[x for x in list_8 if x not in list_0_new_org]
    dict_8=dict(zip(list_8,['SOTF_NA_Rewards_Active012_NA']*len(list_8)))
    
    # 9 SOTF_NA_Rewards_Lapsed1948_NA
    list_9=[x for x in list_names if 'SOTF' in x and 'Lapsed1948' in x]
    list_9=[x for x in list_9 if x not in list_0_new_org]
    dict_9=dict(zip(list_9,['SOTF_NA_Rewards_Lapsed1948_NA']*len(list_9)))
    
    # 10 Legacy_NA_Rewards_Lapsed1318_NA
    list_10=[x for x in list_names if 'Legacy' in x and 'Lapsed1318' in x]
    list_10=[x for x in list_10 if x not in list_0_new_org]
    dict_10=dict(zip(list_10,['Legacy_NA_Rewards_Lapsed1318_NA']*len(list_10)))
     
    # 11 Legacy_NA_Rewards_Active012_NA
    list_11=[x for x in list_names if 'Legacy' in x and 'Active' in x]
    list_11=[x for x in list_11 if x not in list_0_new_org]
    dict_11=dict(zip(list_11,['Legacy_NA_Rewards_Active012_NA']*len(list_11)))
    
    # 12 Legacy_NA_Rewards_Lapsed1948_NA
    list_12=[x for x in list_names if 'Legacy' in x and 'Lapsed1948' in x]
    list_12=[x for x in list_12 if x not in list_0_new_org]
    dict_12=dict(zip(list_12,['Legacy_NA_Rewards_Lapsed1948_NA']*len(list_12)))
    
    # 13 Aud_21mm_Goal_Signed_No_Purchase
    list_13=[x for x in list_names if x=='T_aud_21_goal_1_new_sign_up_no_purchase_0_11_months']
    dict_13=dict(zip(list_13,['Aud_Rewards_21mm_Goal_Signed_No_Purchase']*len(list_13)))
    
    # 14 Aud_21mm_Goal_X_to_12
    list_14=[x for x in list_names if x=='T_aud_21_goal_2_X_to_12']
    dict_14=dict(zip(list_14,['Aud_Rewards_21mm_Goal_X_to_12']*len(list_14)))
    
    # 15 Aud_21mm_Goal_13_to_18
    list_15=[x for x in list_names if x=='T_aud_21_goal_3_13_to_18']
    dict_15=dict(zip(list_15,['Aud_Rewards_21mm_Goal_13_to_18']*len(list_15)))
    
    # 16 Aud_21mm_Goal_19_to_24
    list_16=[x for x in list_names if x=='T_aud_21_goal_4_19_to_24']
    dict_16=dict(zip(list_16,['Aud_Rewards_21mm_Goal_19_to_24']*len(list_16)))
    
    # 17 T_aud_21_goal_5_25_to_30
    list_17=[x for x in list_names if x=='T_aud_21_goal_5_25_to_30']
    dict_17=dict(zip(list_17,['Aud_Rewards_21mm_Goal_25_to_30']*len(list_17)))
    
    # 18 T_Christmas_360_Active
    list_18=[x for x in list_names if x=='T_Christmas_360_Active']
    dict_18=dict(zip(list_18,['T_Rewards_Christmas_360_Active']*len(list_18)))
    
    # 19 T_Christmas_360_Lapsed
    list_19=[x for x in list_names if x=='T_Christmas_360_Lapsed']
    dict_19=dict(zip(list_19,['T_Rewards_Christmas_360_Lapsed']*len(list_19)))
    
    # 20 T_Seasonal362363_Active
    list_20=[x for x in list_names if x=='T_Seasonal362363_Active']
    dict_20=dict(zip(list_20,['T_Rewards_Seasonal362363_Active']*len(list_20)))
    
    # 21 T_Seasonal362363_Lapsed
    list_21=[x for x in list_names if x=='T_Seasonal362363_Lapsed']
    dict_21=dict(zip(list_21,['T_Rewards_Seasonal362363_Lapsed']*len(list_21)))
    
    # 22 Sign_Up_No_Purchase_7_to_21_Days_Test_NEW
    list_22=[x for x in list_names if x=='Sign_Up_No_Purchase_7_to_21_Days_Test_NEW']
    dict_22=dict(zip(list_22,['Sign_Up_No_Purchase_Rewards_7_to_21_Days_Test_NEW']*len(list_22)))
    
    # 22 Sign_Up_No_Purchase_7_to_21_Days_Test_NEW
    list_22=[x for x in list_names if x in ['Sign_Up_No_Purchase_7_to_21_Days_Test_NEW']]
    dict_22=dict(zip(list_22,['Sign_Up_No_Purchase_Rewards_7_to_21_Days_Test_NEW']*len(list_22)))
    
    # 23 4 sets of look alike seasonals
    list_23=[x for x in list_names if x in ['Lookalike (US, 1%) - T_Christmas_360_Active','Lookalike (US, 1%) - T_Christmas_360_Lapsed',
                                           'Lookalike (US, 1%) - T_Seasonal362363_Active','Lookalike (US, 1%) - T_Seasonal362363_Lapsed']]
    dict_23=dict(zip(list_23,list_23))
    
    # 24 10 off 20 
    list_24=[x for x in list_names if "20201102" in x]
    names_24=["aud_Rewards_10_off_20_"+x.split("_")[1] for x in list_24]
    dict_24=dict(zip(list_24,names_24))
    
    # 25 2020Q4 PYD December audience
    list_25=[x for x in list_names if "aud_12162020_PYD_Active_MS_Hashed_20201215020409" in x]
    names_25=["PYD_2020Q4_December_Audience"]
    dict_25=dict(zip(list_25,names_25))

    
    for i in range(-1,25):
        k=i+1
        dict_overall.update(locals()['dict_%i'%k])
    
    # check 
    list_already_allocated=list(dict_overall.keys())
    if len(list_already_allocated)==df_FB_aud_id_name.shape[0]:
        print('all values of the old name are assigned new categories')
        df_FB_aud_id_name['name']=df_FB_aud_id_name['orgname'].apply(lambda x: dict_overall[x])
        return df_FB_aud_id_name,[]
    elif len(list_already_allocated)<df_FB_aud_id_name.shape[0]:
        list_names_missing=[x for x in list_names if x not in list(dict_overall.keys())]
        print("list of names not allocated, renaming as itself: \n %s"%str(list_names_missing))
        df_FB_aud_id_name['name']=df_FB_aud_id_name['orgname'].apply(lambda x: dict_overall[x] if x in dict_overall.keys() else x)
        return df_FB_aud_id_name,list_names_missing
    else:
        raise ValueError("function allocated results dont match, go to check the missing or overlap")

In [16]:
# Add in the new App audience into the categories

dict_categories={'Prospecting 1':['Interests','Shoppers2','Competitive','Life Stage','Women2554withkids',
                                'Big Heros','Mobile Vulnerable Competitors','Mobile Visitors',
                                'Mobile Vicinity','Mobile Non-furniture Specific Competitors'],
                'Prospecting 2':['LAL','Retargeting','Lookalike (US, 1%) - T_Christmas_360_Active','Lookalike (US, 1%) - T_Christmas_360_Lapsed',
                                 'Lookalike (US, 1%) - T_Seasonal362363_Active','Lookalike (US, 1%) - T_Seasonal362363_Lapsed'],
                'SOTF CRM':['T_Rewards_Seasonal362363_Lapsed','T_Rewards_Seasonal362363_Active','T_Rewards_Christmas_360_Lapsed',
                            'T_Rewards_Christmas_360_Active',
                            'Sign_Up_No_Purchase_Rewards_7_to_21_Days_Test_NEW',
                            '2020Q4_Rewards_aud_T_SignUpNoPurchase_2_12', '2020Q4_Rewards_aud_T_SOTF_Lapsed2530_Unknown', '2020Q4_Rewards_aud_T_SOTF_Lapsed1924_Unknown', 
                            '2020Q4_Rewards_aud_T_SOTF_Lapsed1318_L', '2020Q4_Rewards_aud_T_SOTF_ActiveX12_M', '2020Q4_Rewards_aud_T_SOTF_ActiveX12_L', 
                            '2020Q4_Rewards_aud_T_SOTF_Active0X_M', '2020Q4_Rewards_aud_T_SOTF_Active0X_L', '2020Q4_Rewards_aud_T_SOTF_Active0X_H',
                            'aud_Rewards_10_off_20_Online Signup MS B',
                            'aud_Rewards_10_off_20_Online Signup MS A',
                            'aud_Rewards_10_off_20_13-18 Month Lapsed MS B',
                            'aud_Rewards_10_off_20_13-18 Month Lapsed MS A',
                            'aud_Rewards_10_off_20_10-12 Month About to Lapse MS B',
                            'aud_Rewards_10_off_20_10-12 Month About to Lapse MS A',
                            'PYD_2020Q4_December_Audience'
                           ],
                'LEGACY CRM':['T_Rewards_Seasonal362363_Lapsed','T_Rewards_Seasonal362363_Active','T_Rewards_Christmas_360_Lapsed',
                              'T_Rewards_Christmas_360_Active',
                              'Sign_Up_No_Purchase_Rewards_7_to_21_Days_Test_NEW',
                              '2020Q4_Rewards_aud_T_SignUpNoPurchase_2_12',
                              '2020Q4_Rewards_aud_T_Legacy_Lapsed2530_Unknown', '2020Q4_Rewards_aud_T_Legacy_Lapsed1924_Unknown', '2020Q4_Rewards_aud_T_Legacy_Lapsed1318_L', 
                              '2020Q4_Rewards_aud_T_Legacy_ActiveX12_M', '2020Q4_Rewards_aud_T_Legacy_ActiveX12_L', '2020Q4_Rewards_aud_T_Legacy_Active0X_M', 
                              '2020Q4_Rewards_aud_T_Legacy_Active0X_L', '2020Q4_Rewards_aud_T_Legacy_Active0X_H',
                              'aud_Rewards_10_off_20_Online Signup MS B',
                              'aud_Rewards_10_off_20_Online Signup MS A',
                              'aud_Rewards_10_off_20_13-18 Month Lapsed MS B',
                              'aud_Rewards_10_off_20_13-18 Month Lapsed MS A',
                              'aud_Rewards_10_off_20_10-12 Month About to Lapse MS B',
                              'aud_Rewards_10_off_20_10-12 Month About to Lapse MS A',
                              'PYD_2020Q4_December_Audience'
                             ]}

l_qc=[]
for v in dict_categories.values():
    l_qc+=v
print(len(set(l_qc)))

45


In [17]:
df, list_staysame_yet=func_categorize_aud(df)

all values of the old name are assigned new categories


In [18]:
df_output_to_SQL=pd.DataFrame()
for i, row in df.iterrows():
    aud_app_name=row['name']
    aud_FB_original_name=row['orgname']
    aud_FB_id=row['id']
    
    list_cat=[]
    for key, value in dict_categories.items():
        if aud_app_name in value:
            list_cat.append(key)
    df_new=pd.DataFrame({"id":[aud_FB_id]*len(list_cat),
                         "orgname":[aud_FB_original_name]*len(list_cat),
                         "name":[aud_app_name]*len(list_cat),
                         "catagory":list_cat,
                        },index=range(len(list_cat)))
    df_output_to_SQL=df_output_to_SQL.append(df_new)
            
print(df.shape,df_output_to_SQL.shape)

(44, 3) (57, 4)


In [21]:
df_output_to_SQL.to_csv(folder_csv_output+"camap_%s.csv"%BigLotsAccount_id.split('_')[1],index=False)

In [22]:
# QC
df_output_to_SQL[df_output_to_SQL['name']=="PYD_2020Q4_December_Audience"]

,id,orgname,name,catagory
0,23846130655090265,aud_12162020_PYD_Active_MS_Hashed_20201215020409,PYD_2020Q4_December_Audience,SOTF CRM
1,23846130655090265,aud_12162020_PYD_Active_MS_Hashed_20201215020409,PYD_2020Q4_December_Audience,LEGACY CRM


# BigLotsDcart10_id="act_2449979771956630"

In [23]:
# 2 BigLotsDcart10_id="act_2449979771956630"

previous_list=dict_previous_json_config[BigLotsDcart10_id]['aud_list']

df=func_fetch_df_names(BigLotsDcart10_id)


In [24]:
list_to_remove=[]
#
list_to_remove_1=[x for x in previous_list if "T_aud_21_goal_" in x]
list_to_remove.extend(list_to_remove_1)
#
list_to_remove_2=[x for x in previous_list if "2020Q3" in x]
list_to_remove.extend(list_to_remove_2)
list_to_remove

[]

In [25]:
for i in list_to_remove:
    try:
        previous_list.remove(i)
        print("%s removed from previous_list"%i)
    except:
        print("%s not in previous_list"%i)
        pass
    

In [26]:
# list_new_names=[x for x in df['name'].tolist() if "aud_T_" in x]
# list_new_names=[x for x in list_new_names if "control" not in x.lower()]
# list_new_names.sort()


# Just apply the same new list above, because they should be the same
print(list_new_names)

['aud_12162020_PYD_Active_MS_Hashed_20201215020409']


In [27]:
list_updated=list(set(previous_list+list_new_names))
dict_account_2_updated={BigLotsDcart10_id:{'date_update_on':str(datetime.datetime.now().date()),'aud_list':list_updated}}
df=df[df['name'].isin(list_updated)]
len(list_updated),len(df.index)

(44, 44)

In [28]:
# apply func_categorize_aud above

In [29]:
df, list_staysame_yet=func_categorize_aud(df)

all values of the old name are assigned new categories


In [31]:
df_output_to_SQL.to_csv(folder_csv_output+"camap_%s.csv"%BigLotsDcart10_id.split('_')[1],index=False)

In [33]:
df_output_to_SQL[df_output_to_SQL['name']=="PYD_2020Q4_December_Audience"]

,id,orgname,name,catagory
0,23846130655090265,aud_12162020_PYD_Active_MS_Hashed_20201215020409,PYD_2020Q4_December_Audience,SOTF CRM
1,23846130655090265,aud_12162020_PYD_Active_MS_Hashed_20201215020409,PYD_2020Q4_December_Audience,LEGACY CRM


In [35]:
len(dict_account_1_updated)

1

In [36]:
dict_updated_json_config.update(dict_account_1_updated)
dict_updated_json_config.update(dict_account_2_updated)

with open(folder_archieved_list+"archived_on_%s.json"%str(datetime.datetime.now().date()),'w') as outfile:
    json.dump(dict_previous_json_config,outfile)

with open(latest_json_list,'w') as outfile:
    json.dump(dict_updated_json_config,outfile)
    

In [37]:
# Right Category of the targeting
df_audience_category=pd.read_csv(folder_csv_output+"camap_%s.csv"%BigLotsDcart10_id.split('_')[1],dtype=str)
df=pd.read_csv(folder_csv_output+"camap_%s.csv"%BigLotsAccount_id.split('_')[1],dtype=str)
print(df_audience_category.shape)
df_audience_category=df_audience_category.append(df).drop_duplicates()
print(df_audience_category.shape)
df_audience_category=df_audience_category[['name']].drop_duplicates()
print(df_audience_category.shape)


(57, 4)
(57, 4)
(39, 1)


In [38]:
print(df_audience_category['name'].tolist())

['PYD_2020Q4_December_Audience', 'aud_Rewards_10_off_20_Online Signup MS B', 'aud_Rewards_10_off_20_Online Signup MS A', 'aud_Rewards_10_off_20_13-18 Month Lapsed MS B', 'aud_Rewards_10_off_20_13-18 Month Lapsed MS A', 'aud_Rewards_10_off_20_10-12 Month About to Lapse MS B', 'aud_Rewards_10_off_20_10-12 Month About to Lapse MS A', '2020Q4_Rewards_aud_T_SignUpNoPurchase_2_12', '2020Q4_Rewards_aud_T_SOTF_Lapsed2530_Unknown', '2020Q4_Rewards_aud_T_SOTF_Lapsed1924_Unknown', '2020Q4_Rewards_aud_T_SOTF_Lapsed1318_L', '2020Q4_Rewards_aud_T_SOTF_ActiveX12_M', '2020Q4_Rewards_aud_T_SOTF_ActiveX12_L', '2020Q4_Rewards_aud_T_SOTF_Active0X_M', '2020Q4_Rewards_aud_T_SOTF_Active0X_L', '2020Q4_Rewards_aud_T_SOTF_Active0X_H', '2020Q4_Rewards_aud_T_Legacy_Lapsed2530_Unknown', '2020Q4_Rewards_aud_T_Legacy_Lapsed1924_Unknown', '2020Q4_Rewards_aud_T_Legacy_Lapsed1318_L', '2020Q4_Rewards_aud_T_Legacy_ActiveX12_M', '2020Q4_Rewards_aud_T_Legacy_ActiveX12_L', '2020Q4_Rewards_aud_T_Legacy_Active0X_M', '2020Q4_R

In [39]:
[x for x in l_qc if x not in df_output_to_SQL['name'].tolist()]

# Some audience categorized by Spencer, and confirmed with her on Dec 15, 2020 that it can be ignored on our side

['Interests',
 'Shoppers2',
 'Competitive',
 'Life Stage',
 'Women2554withkids',
 'Big Heros']

In [40]:
# 1 record in the jupyter cell of deleted audience

# removed from 'SOTF CRM'
'SOTF_NA_Rewards_Active012_NA','SOTF_NA_Rewards_Lapsed1318_NA','SOTF_NA_Rewards_Lapsed1948_NA',
'aud_BL_2020Q3_PYD_Oct_Rewards_SUNP12','aud_BL_2020Q3_PYD_Oct_Rewards_Lapsed1324','aud_BL_2020Q3_PYD_Oct_Rewards_Act12_B',
'aud_BL_2020Q3_PYD_Oct_Rewards_Act12_A3','aud_BL_2020Q3_PYD_Oct_Rewards_Act12_A2','aud_BL_2020Q3_PYD_Oct_Rewards_Act12_A1',
'Aud_Rewards_21mm_Goal_25_to_30','Aud_Rewards_21mm_Goal_19_to_24',
'Aud_Rewards_21mm_Goal_13_to_18','Aud_Rewards_21mm_Goal_X_to_12','Aud_Rewards_21mm_Goal_Signed_No_Purchase',

('Aud_Rewards_21mm_Goal_13_to_18',
 'Aud_Rewards_21mm_Goal_X_to_12',
 'Aud_Rewards_21mm_Goal_Signed_No_Purchase')

In [41]:
# 1 record in the jupyter cell of deleted audience

# removed from 'LEGACY CRM'
'Legacy_NA_Rewards_Active012_NA','Legacy_NA_Rewards_Lapsed1318_NA','Legacy_NA_Rewards_Lapsed1948_NA',
'aud_BL_2020Q3_PYD_Oct_Rewards_SUNP12','aud_BL_2020Q3_PYD_Oct_Rewards_Lapsed1324','aud_BL_2020Q3_PYD_Oct_Rewards_Act12_B',
'aud_BL_2020Q3_PYD_Oct_Rewards_Act12_A3','aud_BL_2020Q3_PYD_Oct_Rewards_Act12_A2','aud_BL_2020Q3_PYD_Oct_Rewards_Act12_A1',
'Aud_Rewards_21mm_Goal_25_to_30','Aud_Rewards_21mm_Goal_19_to_24',
'Aud_Rewards_21mm_Goal_13_to_18','Aud_Rewards_21mm_Goal_X_to_12','Aud_Rewards_21mm_Goal_Signed_No_Purchase',

('Aud_Rewards_21mm_Goal_13_to_18',
 'Aud_Rewards_21mm_Goal_X_to_12',
 'Aud_Rewards_21mm_Goal_Signed_No_Purchase')

In [42]:
a=df_audience_category['name'].tolist()
[x for x in a if x not in l_qc]

[]

In [43]:
latest_category_mapping

'/mnt/clients/juba/hqjubaapp02/sharefolder/Docs/Audience_uploading_scripts/Facebook/Audience_name_list_On_App/latest_category_mapping.json'

In [44]:
with open(latest_category_mapping,'w') as outfile:
    json.dump(dict_categories,outfile)
    

In [45]:
###update exclude

# The function in the app is an account level suppresion, so no need to frequently update here
# Ask Spencer if need to run below

In [30]:
previous_list=dict_previous_json_config[BigLotsAccount_id]['aud_list']

df=func_fetch_df_names(BigLotsAccount_id)
df_control = df[df['name'].isin(["all_BL_rewards_members_email",
                          "Control_2020Q3_segment",
                          "CCPA_suppression",
                          'Seasonal_control_overall',
                          'aud_21_goal_control_overal',
                          'Sign_Up_No_Purchase_7_to_21_Days_Control_NEW'])]


In [31]:
dict_control={'CCPA_suppression':'control1',
             'Control_2020Q3_segment':'control1',
             'all_BL_rewards_members_email':'control2',
             'Seasonal_control_overall':'control3_seasonal',
             'aud_21_goal_control_overal':'control4_21mm',
             'Sign_Up_No_Purchase_7_to_21_Days_Control_NEW':'control5_new7_21'}

In [33]:
df_control['orgname'] = df_control['name']
df_control['name'] = df_control['orgname'].apply(lambda x: dict_control[x])
df_control

/home/jian/.local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/jian/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,id,name,orgname
5,23845432420230265,control3_seasonal,Seasonal_control_overall
6,23845430128380265,control4_21mm,aud_21_goal_control_overal
24,23845393338810265,control5_new7_21,Sign_Up_No_Purchase_7_to_21_Days_Control_NEW
86,23844984551590265,control1,Control_2020Q3_segment
206,23844601952190265,control1,CCPA_suppression
559,23844525980280265,control2,all_BL_rewards_members_email


In [37]:
df_control.to_csv(folder_csv_output+"remove_"+BigLotsAccount_id.split('_')[1]+'.csv',index = False)
df_control.to_csv(folder_csv_output+"remove_"+BigLotsDcart10_id.split('_')[1]+'.csv',index = False)

In [28]:
a=['T_Christmax_360_Lapsed','T_Christmax_360_Active','T_Seasonalall_3_Active','T_Seasonalall_3_Lapsed']
b=['T_Christmas_360_Lapsed','T_Christmas_360_Active','T_Seasonal362363_Active','T_Seasonal362363_Lapsed']
to_ruoyi=dict(zip(a,b))
to_ruoyi

{'T_Christmax_360_Lapsed': 'T_Christmas_360_Lapsed',
 'T_Christmax_360_Active': 'T_Christmas_360_Active',
 'T_Seasonalall_3_Active': 'T_Seasonal362363_Active',
 'T_Seasonalall_3_Lapsed': 'T_Seasonal362363_Lapsed'}